In [ ]:
import numpy as np
from random import choice 

size = 9
x = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9], 
              [4, 5, 6, 7, 8, 9, 1, 2, 3], 
              [7, 8, 9, 1, 2, 3, 4, 5, 6], 
              [2, 1, 4, 3, 6, 5, 8, 9, 7], 
              [3, 6, 5, 8, 9, 7, 2, 1, 4],
              [8, 9, 7, 2, 1, 4, 3, 6, 5], 
              [5, 3, 1, 6, 4, 2, 9, 7, 8],
              [6, 4, 2, 9, 7, 8, 5, 3, 1], 
              [9, 7, 8, 5, 3, 1, 6, 4, 2]])

def gen_upper_left_x(size=9):
    out = list()
    for i in range(0, size):
        out += [i]*(size-i)
    return np.array(out)

def gen_upper_left_y(size=9):
    out = list()
    for i in range(0, size):
        out += list(np.arange(0, size-i))
    return np.array(out)

def gen_lower_right_x(size=9):
    out = list()
    for i in range(0, size):
        out += list(np.arange(i, size))
    return np.array(out)

def gen_lower_right_y(size=9):
    out = list()
    for i in range(0, size):
        out += [size-i-1]*(size-i)
    return np.array(out)

def gen_ul_indices(size=9):
    return [(x, y) for x, y in zip(gen_upper_left_x(size), gen_upper_left_y(size))]

def gen_ll_indices(size=9):
    return [(x, y) for x, y in zip(np.tril_indices(size)[0], np.tril_indices(size)[1])]

def gen_ur_indices(size=9):
    return [(x, y) for x, y in zip(np.triu_indices(size)[0], np.triu_indices(size)[1])]

def gen_lr_indices(size=9):
    return [(x, y) for x, y in zip(gen_lower_right_x(size), gen_lower_right_y(size))]


def random_corner(size):
    return choice([(0, 0), (0, size-1), (size-1, 0), (size-1, size-1)])

def get_adjacent(tri_ind):
    adjacent = dict()
    for idx in range(0, len(tri_ind)):
        i = tri_ind[idx]
        idx_list = tri_ind[:idx] + tri_ind[idx + 1:]
        adjacent.update({i: [j for j in idx_list if (abs(j[0] - i[0]) == 1 and j[1] - i[1] == 0) or (abs(j[1] - i[1]) == 1 and j[0] - i[0] == 0)]})
    return adjacent

def gen_random_region(x, size):
    # start w/ random corner
    corner = random_corner(size)
    # pick a random direction to start, generate available indices and their adjacent indices
    if corner == (0, 0):
        x_start = (1, 0)
        y_start = (0, 1)
        tri_ind = get_adjacent(gen_ul_indices(size))
    elif corner == (0, size-1):
        x_start = (1, size-1)
        y_start = (0, size-2)
        tri_ind = get_adjacent(gen_ur_indices(size))
    elif corner == (size-1, 0):
        x_start = (size-2, 0)
        y_start = (size-1, 1)
        tri_ind = get_adjacent(gen_ll_indices(size))
    else:
        x_start = (size-2, size-1)
        y_start = (size-1, size-2)
        tri_ind = get_adjacent(gen_lr_indices(size))
    # start out with no exclusions
    exclude = list()
    # random starting checks
    start_choices = [x_start, y_start]
    start = choice(start_choices)
    start_choices = [choice for choice in start_choices if choice != start]
    # start != corner, so move on 
    # start building region (of indices, and of values)
    region_i = [corner, start]
    region_v = [x[corner], x[start]]
    while len(region_v) < size:
        next = 2
        # get adjacent
        next_ind = [i for i in tri_ind[region_i[next - 1]] if i not in region_i and i not in exclude and x[i] not in region_v]
        if len(next_ind) == 0:
            if len(region_i) > 2:
                exclude.append(region_i.pop())
                region_v.pop()
                next += -1
            elif len(region_i) == 2 and len(start_choices) == 1:
                start = start_choices[0]
                region_i = [corner, start]
                region_v = [x[corner], x[start]]
                print(region_i, region_v)
            else:
                raise Exception('There are no possible solutions given this arrangement')
        else:
            next += 1
            next_ind = choice(next_ind)
            region_i.append(next_ind)
            region_v.append(x[next_ind])
        
    
print(x)
gen_random_region(x, size)